# Market basket analysis

This project aims to perform market basket analysis for [groceries dataset](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset) downloaded from Kaggle.

Project aims: Utilising Apriori alorithm to develop assiociation rules

## Data preprocessing

Importing numpy and pandas.

In [1]:
import pandas as pd
import numpy as np

Read the CSV file into a dataframe

In [2]:
df = pd.read_csv('groceries_dataset.csv')
df

,Member_number,Date,itemDescription
0,1808,21/07/2015,tropical fruit
1,2552,5/01/2015,whole milk
2,2300,19/09/2015,pip fruit
3,1187,12/12/2015,other vegetables
4,3037,1/02/2015,whole milk
...,...,...,...
38760,4471,8/10/2014,sliced cheese
38761,2022,23/02/2014,candy
38762,1097,16/04/2014,cake bar
38763,1510,3/12/2014,fruit/vegetable juice


Checking for duplicate rows based on the 'Member_number' and 'Date'.

In [3]:
duplicate_rows = df[df.duplicated(subset=['Member_number', 'Date', 'itemDescription'], keep=False)]
duplicate_rows.sort_values(by=['Member_number','Date'])

,Member_number,Date,itemDescription
33098,1003,27/02/2014,rolls/buns
37649,1003,27/02/2014,rolls/buns
15099,1005,9/01/2014,rolls/buns
31248,1005,9/01/2014,rolls/buns
7532,1006,14/06/2015,frankfurter
...,...,...,...
24043,4981,10/01/2015,margarine
8109,4988,29/10/2015,rolls/buns
24258,4988,29/10/2015,rolls/buns
33585,4992,24/02/2014,margarine


As there are duplicated rows, indicating multiple purchases of the same items, we do not require the quantity for our calculations. Therefore we need to remove the duplicated rows.

In [4]:
df = df.drop_duplicates()

To use the Apriori algorithm, we need to convert the dataset into true and false (1 or 0), with purchased items as columns. We process the data as follows.

In [5]:
# Assign a unique transaction ID to each transaction based on Member_number and Date
df.loc[:, 'TransactionID']=df.groupby(['Member_number', 'Date']).ngroup()

# Sort the DataFrame by Member_number and Date, then drop these columns
df=df.sort_values(by=['Member_number','Date']).drop(columns=['Member_number','Date'])

# Add a 'Present' column with all values set to True
df['Present'] = True

C:\Users\kdsha\AppData\Local\Temp\ipykernel_62544\219977385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'TransactionID']=df.groupby(['Member_number', 'Date']).ngroup()


Then, we convert the data using pivot.

In [6]:
# Pivot the df to convert it into a binary matrix where rows represent transactions, 
# columns represent item descriptions, and values indicate whether an item is present in a transaction
pivoted = df.pivot(index='TransactionID', columns='itemDescription', values='Present').fillna(False)

# Remove the column name to improve readability
pivoted.columns.name = None

# Remove the index name to improve readability
pivoted = pivoted.rename_axis(None, axis=1)

# Return the pivoted DataFrame
pivoted

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
TransactionID,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Association Rule Mining with Apriori Algorithm

Import required libraries.

In [7]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [8]:
from mlxtend.frequent_patterns import apriori, association_rules

Next we mine frequent itemsets using the Apriori algorithm with the pivoted DataFrame.

In [9]:
# Specify a minimum support threshold of 0.002 and use item names as column names
frequent_itemsets_new = apriori(pivoted, min_support=0.002, use_colnames=True)

In [10]:
# Print frequent itemsets
print(frequent_itemsets_new)

      support                          itemsets
0    0.004010           (Instant food products)
1    0.021386                        (UHT-milk)
2    0.008087                   (baking powder)
3    0.033950                            (beef)
4    0.021787                         (berries)
..        ...                               ...
325  0.002606             (whole milk, waffles)
326  0.004611  (whipped/sour cream, whole milk)
327  0.002941      (whipped/sour cream, yogurt)
328  0.003141         (whole milk, white bread)
329  0.011161              (whole milk, yogurt)

[330 rows x 2 columns]


Generate association rules from frequent itemsets using the lift metric.

In [11]:
# Specify a minimum lift threshold of 1.0
rules = association_rules(frequent_itemsets_new, metric='lift', min_threshold=1.0)

Print association rules.

In [12]:
print(rules)

                antecedents              consequents  antecedent support  \
0                 (berries)       (other vegetables)            0.021787   
1        (other vegetables)                (berries)            0.122101   
2                 (sausage)           (bottled beer)            0.060349   
3            (bottled beer)                (sausage)            0.045312   
4             (canned beer)            (brown bread)            0.046916   
5             (brown bread)            (canned beer)            0.037626   
6             (brown bread)                 (pastry)            0.037626   
7                  (pastry)            (brown bread)            0.051728   
8               (chocolate)             (rolls/buns)            0.023592   
9              (rolls/buns)              (chocolate)            0.110005   
10           (citrus fruit)                 (yogurt)            0.053131   
11                 (yogurt)           (citrus fruit)            0.085879   
12          